In [2]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [547]:
class Image():
    ''' 
    Class for store images.
    '''

    FILEPATH = 'ece661_pics\\hw6_image\\'

    def __init__(self, name, savename):
        self.name = name
        self.savename = savename
        self.load_images()
        

    def load_images(self):
        self.image = cv.imread(self.FILEPATH + self.name)
        self.image_gray = cv.cvtColor(self.image, cv.COLOR_BGR2GRAY)
        self.image_red = self.image[:,:, 2]
        self.image_green = self.image[:,:, 1]
        self.image_blue = self.image[:,:, 0]
        
    
    def load_variance(self):
        self.load_images()
        self.image_3 = self.compute_variance(3)
        self.image_5 = self.compute_variance(5)
        self.image_7 = self.compute_variance(7)

    def compute_variance(self, window):
        ''' 
            Compute variance of image by window size.
        '''
        h, w = self.image_gray.shape
        width = int((window - 1) / 2)
        variance = np.zeros((h, w))
        for j in range(h):
            for i in range(w):
                kernal = self.image_gray[j-width : j+width+1, i-width : i+width+1]
                if kernal.size == 0:
                    variance[j][i] = 0
                else:
                    variance[j][i] = np.var(kernal)
        # Normalize to 0-255
        variance = (variance * 255 / variance.max())

        return variance.astype(np.uint8)

    def show_image(self, color=None):
        if color is None:
            plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))
        elif color == 'gray':
            plt.imshow(self.image_gray, cmap='gray')
        elif color == 'red':
            plt.imshow(self.image_red, cmap='gray')
        elif color == 'blue':
            plt.imshow(self.image_blue, cmap='gray')
        elif color == 'green':
            plt.imshow(self.image_green, cmap='gray')      

img1 = Image('pigeon.jpeg', 'hw6_1')   
img2 = Image('Red-Fox.jpg', 'hw6_2') 
img3 = Image('cat.jpg', 'hw6_3')

In [588]:
class Otsu():
    '''
        Class for implementing Otsu algorithm.
    '''

    FILEPATH = 'ece661_pics\\hw6_image\\'

    def __init__(self):
        pass

    def find_threshore(self, layer):
        '''
            Find the threshore of particular image layer.
        '''
        # Ignore pixel intensity = 0 for Otsu iteration
        [hist, bin_edges] = np.histogram(layer, bins=np.arange(1, 257), density=True)
        prob_wieght = hist * bin_edges[:-1]
        mean = np.sum(prob_wieght) # mean of the whole layer
        k = 0
        var_max = 0
        w0 = 0
        prob_wieght_sum = 0
        # Compute parameters
        for i in range(hist.size):
            w0 += hist[i]
            w1 = 1 - w0
            prob_wieght_sum += prob_wieght[i]
            u0 = prob_wieght_sum / w0
            if w1 != 0:
                u1 = (mean - prob_wieght_sum) / w1
                var = w0 * w1 * (u0 - u1)**2
                if var > var_max:
                    var_max = var
                    k = i

        return k+1

    def get_mask(self, layer, threshore, image_name, layer_name, higher=True):
        '''
            Get array of 0 and 1 as the mask of the layer.
        '''
        if higher:
            mask = (layer > threshore).astype(np.uint8)
        else:
            mask = (layer < threshore).astype(np.uint8)
        savename = f'{self.FILEPATH}{image_name}_mask_{layer_name}.png'
        cv.imwrite(savename, mask*255)

        return mask

    def find_contour(self, mask, image_name):
        '''
            Get the contour mask
        '''
        contour = np.zeros(mask.shape).astype(np.uint8)
        ones = np.argwhere(mask == 1)
        for one in ones:
            x = one[1]
            y = one[0]
            window = mask[y-1:y+2, x-1:x+2]
            if np.sum(window) < 9 and window.size == 9:
                contour[y][x] = 1
        
        savename = f'{self.FILEPATH}{image_name}_contour.png'
        cv.imwrite(savename, contour*255)

        return contour

    def color_segmentaion(self, image, iterations, morphs):
        '''
            Implement segmetation with each RGB layer.
        '''
        for iteration in range(iterations):
            morph_size = morphs[iteration]
            filename = f'{image.savename}_color_iter{iteration+1}'
            # Find the threshore
            k_red = self.find_threshore(image.image_red)
            k_green = self.find_threshore(image.image_green)
            k_blue = self.find_threshore(image.image_blue)
            # Get masks for each layer and combine them together
            mask_red = self.get_mask(image.image_red, k_red, filename, 'red')
            mask_green = self.get_mask(image.image_green, k_green, filename, 'green')
            mask_blue = self.get_mask(image.image_blue, k_blue, filename, 'blue')
            mask = mask_red * mask_green * mask_blue
            savename = f'{self.FILEPATH}{filename}_mask_premorph.png'
            cv.imwrite(savename, mask*255)
            # Closing holes
            kernel = np.ones((morph_size, morph_size),np.uint8)
            mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
            savename = f'{self.FILEPATH}{filename}_mask.png'
            cv.imwrite(savename, mask*255)

            contour = self.find_contour(mask, filename)
            for i in range(3):
                image.image[:, :, i] *= mask
            savename = f'{self.FILEPATH}{filename}.png'
            cv.imwrite(savename, image.image)
    
    def feature_segmentation(self, image, iterations, morphs):
        '''
            Implement segmetation by feature.
        '''
        img = image.image_gray
        for iteration in range(iterations):
            morph_size = morphs[iteration]
            filename = f'{image.savename}_feature_iter{iteration+1}'
            # Find the threshore
            k_3 = self.find_threshore(image.image_3)
            k_5 = self.find_threshore(image.image_5)
            k_7 = self.find_threshore(image.image_7)
            # Get masks for each layer and combine them together
            mask_3 = self.get_mask(image.image_3, k_3, filename, '3', False)
            mask_5 = self.get_mask(image.image_5, k_5, filename, '5', False)
            mask_7 = self.get_mask(image.image_7, k_7, filename, '7', False)
            mask = mask_3 * mask_5 * mask_7
            savename = f'{self.FILEPATH}{filename}_mask_premorph.png'
            cv.imwrite(savename, mask*255)
            # Closing holes
            kernel = np.ones((morph_size, morph_size),np.uint8)
            finalmark = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
            savename = f'{self.FILEPATH}{filename}_mask.png'
            cv.imwrite(savename, mask*255)

            contour = self.find_contour(mask, filename)
            for i in range(3):
                image.image[:, :, i] *= mask
            image.image_3 *= mask
            image.image_5 *= mask
            image.image_7 *= mask
            savename = f'{self.FILEPATH}{filename}.png'
            cv.imwrite(savename, image.image)

otsu = Otsu()

In [589]:
img1.load_images()
otsu.color_segmentaion(img1, 2, [15, 21])

In [596]:
img2.load_images()
otsu.color_segmentaion(img2, 2, [1, 45])

In [591]:
img3.load_images()
otsu.color_segmentaion(img3, 1, [67])

In [592]:
img1.load_variance()
otsu.feature_segmentation(img1, 2, [5, 9])

In [593]:
img2.load_variance()
otsu.feature_segmentation(img2, 1, [5])

In [594]:
img3.load_variance()
otsu.feature_segmentation(img3, 2, [5, 5])